In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U accelerate
!pip install -q -U datasets
!pip install -q -U peft

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, PeftModel, get_peft_model
import torch

In [3]:
base_model = "meta-llama/Meta-Llama-3.1-8B"
weights = "/kaggle/input/llama-3-1-fine-tuning/llama3.1-meetings-finetune/checkpoint-300/"

In [4]:
# Quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [5]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HF_TOKEN")
login(token=secret_value_0)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [6]:
# Loading the model and tokenizer
model = AutoModelForCausalLM.from_pretrained(base_model, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(
    base_model,
    model_max_length=512,
    padding_side="left",
    add_eos_token=True
)
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [7]:
ft_model = PeftModel.from_pretrained(
    model, 
    "/kaggle/input/llama-3-1-fine-tuning/llama3.1-meetings-finetune/checkpoint-300")

In [8]:
transcript = """
Counselor (C): Hi Jane, how are you today?
Jane (J): I’m good, thanks!
C: Great! Let’s talk about your career goals. What’s your background?
J: I’ve been a software developer for two years, but I’m considering a change—maybe product management or UI/UX design.
C: What do you enjoy in your current role, and what don’t you like?
J: I enjoy problem-solving, but the routine and deadlines are stressful. I want something more creative.
C: That makes sense. For product management or UI/UX, you could leverage your tech background. Have you thought about building a portfolio for UI/UX?
J: I’ve seen job listings asking for portfolios, but I’m not sure where to start.
C: Try redesigning apps or websites. Use Figma or Adobe XD, document your process, and include it in your portfolio.
J: That’s helpful. I’ll start practicing.
C: Also, connect with industry professionals and join design communities to expand your network.
J: Got it! Thanks, I’ll start working on that.
C: Great! Let’s meet again in a few weeks to check your progress.
J: Sounds good. Thanks again!
"""

In [10]:
inputs = tokenizer("Summarize the meeting transcript and extract meaningful insights.\n" + transcript, return_tensors="pt")
outputs = ft_model.generate(**inputs, max_new_tokens=300)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1885: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Summarize the meeting transcript and extract meaningful insights.

Counselor (C): Hi Jane, how are you today?
Jane (J): I’m good, thanks!
C: Great! Let’s talk about your career goals. What’s your background?
J: I’ve been a software developer for two years, but I’m considering a change—maybe product management or UI/UX design.
C: What do you enjoy in your current role, and what don’t you like?
J: I enjoy problem-solving, but the routine and deadlines are stressful. I want something more creative.
C: That makes sense. For product management or UI/UX, you could leverage your tech background. Have you thought about building a portfolio for UI/UX?
J: I’ve seen job listings asking for portfolios, but I’m not sure where to start.
C: Try redesigning apps or websites. Use Figma or Adobe XD, document your process, and include it in your portfolio.
J: That’s helpful. I’ll start practicing.
C: Also, connect with industry professionals and join design communities to expand your network.
J: Got it! 